In [1]:
from hpc_env import HPCenv

In [2]:
import gymnasium as gym
import numpy as np
import configparser
import ast
import os

from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

In [3]:
# Load config with explicit path and typed parsing
config = configparser.ConfigParser()
config_path = os.path.join(os.getcwd(), 'config_file', 'config.ini')
config.read(config_path)
MAX_QUEUE_SIZE = config.getint('architecture', 'max_queue_size')
RUN_WIN_LENGTH = config.getint('architecture', 'run_win_length')
GREEN_FORECAST_LENGTH = config.getint('architecture', 'green_forecast_length')

DELAY_TIME_LIST = ast.literal_eval(config.get('architecture', 'delay_time_list'))
DELAY_TIME_LIST_LENGTH = len(DELAY_TIME_LIST)

JOB_FEATURE = config.getint('architecture', 'job_feature')
RUN_FEATURE = config.getint('architecture', 'run_feature')
GREEN_FEATURE_PR_TIMESLOT = config.getint('architecture', 'green_feature_pr_timeslot')
GREEN_FEATURE_CONSTANT = config.getint('architecture', 'green_feature_constant')
MAX_WAIT_N_JOBS = config.getint('architecture', 'max_wait_n_jobs')

# Power / carbon settings (use correct option names)
IDLE_POWER = config.getfloat('power settings', 'idle_power')
NUM_PROCS_PER_NODE = config.getint('power settings', 'procs_per_node')
CARBON_YEAR = config.getint('power settings', 'carbon_year')

EPISODE_LENGTH = config.getint('training', 'episode_length')

## Masking function size

In [4]:
WORKLOAD_PATH = "data/workloads/lublin_256.swf"
env = HPCenv(workload_path=WORKLOAD_PATH)
def mask_fn(env: gym.Env) -> np.ndarray:
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.valid_action_mask()
masking_function = mask_fn(env)

assert (env.action_space_size == len(masking_function))
print("Masking function test passed ✅")

Using constant power: 500.0 watts per processor
Max Allocated Processors: 256 ;max node: 256 ;max procs: 256 ;max execution time: 124707
Masking function test passed ✅


#### Observation state 

In [ ]:
env = HPCenv(workload_path=WORKLOAD_PATH)
env.reset(seed=42,options=None)

predefined_size = env.observation_space._shape
actual_shape = env.build_observation().shape

print("Predefined sized:", predefined_size)
print("Actual observation state size: ", actual_shape)
assert predefined_size == actual_shape
print("Observation state size passed ✅")

Using constant power: 500.0 watts per processor
Max Allocated Processors: 256 ;max node: 256 ;max procs: 256 ;max execution time: 124707
Predefined sized: (1487,)
Actual observation state size:  (1487,)
Observation state size passed ✅
